In [1]:
import datetime
import os
from mdagent import MDAgent

In [2]:
prompt4 = "Simulate 1XQ8 for 1ns at temperate 300K. Then tell me if the secondary structure changed from before the simulation to after."
llm_var = "llama-v3p1-405b-instruct"
tools = "all"
agent = MDAgent(agent_type="Structured", model=llm_var, top_k_tools=tools)
now = datetime.datetime.now()
date = now.strftime("%Y-%m-%d")
print("date and time:",date)
time = now.strftime("%H:%M:%S")
print("time:",time)
print("LLM: ",agent.llm.model_name,"\nTemperature: ",agent.llm.temperature)

date and time: 2024-08-06
time: 10:36:07
LLM:  accounts/fireworks/models/llama-v3p1-405b-instruct 
Temperature:  0.1


In [3]:
agent.run(prompt4)

Num of tokens: 368


> Entering new AgentExecutor chain...
Thought: First, I need to set up and run a simulation of the protein 1XQ8 for 1ns at a temperature of 300K. Then, I will compare the secondary structure of the protein before and after the simulation to see if it has changed.

Action: SetUpandRunFunction

Action Input: 
{
  "pdb_id": "1XQ8",
  "forcefield_files": ["amber99sbildn.xml"],
  "save": true,
  "system_params": {
    "nonbondedMethod": "NoCutoff",
    "constraints": "None",
    "rigidWater": false
  },
  "integrator_params": {
    "integrator_type": "LangevinMiddle",
    "Temperature": "300 * kelvin",
    "Friction": "1.0 / picoseconds",
    "Timestep": "0.002 * picoseconds"
  },
  "simulation_params": {
    "Ensemble": "NVT",
    "Number of Steps": 500000,
    "record_interval_steps": 1000,
    "record_params": ["step", "potentialEnergy", "temperature"]
  }
}<|eom_id|><|start_header_id|>assistant<|end_header_id|>

Thought: First, I need to set up and run a simulation 

({'input': '\n    You are an expert molecular dynamics scientist and\n    your task is to respond to the question or\n    solve the problem to the best of your ability using\n    the provided tools.\n\n    You can only respond with a single complete\n    Thought, Action, Action Input\' format\n    OR a single \'Final Answer\' format.\n\n    Complete format:\n    Thought: (reflect on your progress and decide what " "to do next)\n    Action: (the action name, should be the name of a tool)\n    Action Input: (the input string to the action)\n\n    OR\n\n    Final Answer: (the final response to the original input\n    question, when all steps are complete)\n\n    You are required to use the tools provided,\n    using the most specific tool\n    available for each action.\n    Your final answer should contain all information\n    necessary to answer the question and subquestions.\n    Before you finish, reflect on your progress and make\n    sure you have addressed the question in its entir

In [4]:
registry = agent.path_registry
print(registry.list_path_names_and_descriptions())

No names found. The JSON file is empty or does not contain name mappings.


In [5]:
now = datetime.datetime.now()
date = now.strftime("%Y-%m-%d")
print("date and time:",date)
time = now.strftime("%H:%M:%S")
print("time:",time)

date and time: 2024-08-06
time: 10:36:31


In [6]:
registry = agent.path_registry
paths_and_descriptions = registry.list_path_names_and_descriptions()
print("\n".join(paths_and_descriptions.split(",")))

No names found. The JSON file is empty or does not contain name mappings.


In [7]:
# import mdtraj as md
# import numpy as np

# traj_path = registry.get_mapped_path("rec2_164856")
# top_path = registry.get_mapped_path("top_sim0_164856")
# traj = md.load(traj_path, top=top_path)

# # Compute the secondary structure of the trajectory
# dssp_final = md.compute_dssp(traj[-1], simplified=True)
# dssp_initial = md.compute_dssp(traj[0], simplified=True)

# print("Number of initial sheets: ",len([i for i in dssp_initial[0] if i == 'E']))
# print("Number of initial helices: ",len([i for i in dssp_initial[0] if i == 'H']))
# print("Number of initial coils: ",len([i for i in dssp_initial[0] if i == 'C']))

# print("Number of final sheets: ",len([i for i in dssp_final[0] if i == 'E']))
# print("Number of final helices: ",len([i for i in dssp_final[0] if i == 'H']))
# print("Number of final coils: ",len([i for i in dssp_final[0] if i == 'C']))



# Experiment Result:
### Completed without Exception or TimeOut Errors ✅
### Attempted all necessary steps ❌
### No Hallucination ❌
### Logic make sense ✅
### Correct Answer  ❌